In [1]:
from math import *
import numpy as np
from scipy.special import factorial as np_factorial
np.factorial = np_factorial

Por la naturaleza de incertidumbre, no es posible conocer con exactitud cuántos trabajos llegarían al sistema y qué tipo de servicio van a requerir. Para reducir en la meida de lo posible la espera de los potenciales cuando la demanda de un cierto servicio excede la capacidad del sistema sin que suponga un coste excesivo, es decir, equilibrar el coste del servicio con el coste asociado a la perdida de clientes.

![Sistema de colas](sistemas-de-colas.png)

![Notación de Kendall](kendall.jpg)

Proceso de llegadas: número de llegadas por unidad de tiempo

Proceso de servicio: Tiempo empleado en dar servicio a un trabajo

Diciplina de la cola: ordenen que se da servicio a los trabajos

**N** número de llegadas por unidad de tiempo

**T** tiempo entre dos llegadas consecutivas

**X** tiempo de realización de un servicio

**$\lambda_n$** tasa de llegadas cuando el sistema tiene n clientes

**$\mu_n$** tasa de servicio cuando el sistema tiene n clientes

**$\rho=\frac{\lambda}{c\mu}$** intensidad de trafico o factor de utilización, condición de trafico $\rho < 1$

In [2]:
def cal_p(lambda_, mu, c=1):
    p = lambda_ / mu / c
    print("p is: ", p)
    return p

assert (cal_p(60, 1/0.1, 7) - 0.857) < 0.001

p is:  0.8571428571428571


Paso: número de llegadas atendidas por unidad de tiempo, si el sistema no admite pérdidas, coincide con la tasa de llegadas, **paso máximo** es la capacidad máxima $m\mu$

**$\overline{Q}$** Número medio de trabajos

**$\overline{W}$** Número medio de espera

**$\overline{J}$** Número medio de trabajo en el sistena

**$\overline{R}$** Tiempo medio de respuesta

# Fórmulas de Little

Proporciona una relación $L=\lambda\overline{W}$ entre el número $L$ promedio a largo plazo de los clientes en un sistema estacionario y el tiempo promedio $\overline{W}$ que un cliente pasa al sistema. 

$$
\begin{split}
\overline{J} &= \lambda\overline{R}\\
\overline{Q} &= \lambda\overline{W}\\
\overline{R} &= \overline{X} + \overline{W}\\
\end{split}
$$

# Fórmula de Pollaczek-Khintchine

$$C(X) = \frac{\sqrt{V(X)}}{\overline{X}} = \frac{\sqrt{\overline{X^2}-\overline{X}^2}}{\overline{X}}$$

Si el timepo de servicio **X** tiene una distribución exponencial de parámetro $\mu$

$$
\begin{split}
\overline{X} &= \frac{1}{\mu}\\
V(X) &= \frac{2}{\mu^2}\\
\overline{X^2} &= \frac{2}{\mu^2}\\
\end{split}
$$

In [3]:
def cal_X2_exponential(mu):
    X_2 = 2 / mu**2
    print("X^2 is: ", X_2)
    return X_2

assert (cal_X2_exponential(10) - 0.02) < 0.001

X^2 is:  0.02


Si un sistema con $M/G/1$

$$
\begin{split}
\overline{W} &= \lambda\frac{\overline{X^2}}{2(1-\rho)}\\
\overline{W} &= \rho\frac{\overline{X}(1+C(X)^2)}{2(1-\rho)}
\end{split}
$$

In [4]:
def cal_W_MG1(lambda_, mu, CX=1):
    p = cal_p(lambda_, mu)
    W = p * (1 + CX**2) / (2 * mu * (1 - p))
    print("W is: ", W)
    return W

assert abs(cal_W_MG1(2, 3, 1/4) + 1/3 - 0.6875) < 0.001

p is:  0.6666666666666666
W is:  0.35416666666666663


El mínimo de $\overline{R}$ se alcanza cuando el tiempo de servicio es constante (cola $M/D/1$).

$$\overline{R}=\frac{\rho\overline{X}}{2(1-\rho)}+\overline{X}$$

Si observamos el lote como una única llegada el tiempo medio de espera B se puede obtener a partir de las fórmulas de Pollactek-Khinchine

$$B=\lambda'(b\overline{X})^2\frac{(1+\frac{bV}{(b \overline{X})^2})}{2(1-\lambda' b\overline{X})}$$

El tiempo total de respuesta de un trabajo del lote es

$$\overline{R}=B+\overline{X}\frac{b+1}{2}$$

# Sistemas con prioridad no expulsiva

$\overline{X}_i$ tiempo medio de espera de un trabajo de la clase $i$ se calcula como

$$\overline{W}_i = \overline{S} + D_i + B_i$$

$\overline{S}$ tiempo medio que queda para que se termine el trabajo que actualmente está en el servidor

$$
\overline{S}=\frac{1}{2}\sum_{k=1}^p \lambda_k \overline{X^2_k}
$$

In [5]:
def cal_S(lambdas, X_2):
    S = np.sum(lambdas*X_2, axis=0) / 2
    print("S is: ", S)
    return S

assert abs(cal_S(np.array([0.8, 0.2]), np.array([0.5, 2])) - 0.4) < 0.001

S is:  0.4


$B_i$ tiempo medio de proceso de todos los trabajos de las clases 1 hasta i que ya están en la cola

$D_i$ tiempo medio de proceso de todos los trabajos de las clases 1 hasta $i-1$ que pueden llegar, desde que ha llegado el trabajo hasta que empiece su ejecución.

$$
\begin{split}
U_0 &= 0\\
U_i &= \sum_{k=1}^i \rho_k\\
\end{split}
$$

In [6]:
def cal_U(lambdas, mus):
    p = cal_p(lambdas, mus)
    U = np.array([np.sum(p[:i]) for i in range(p.size + 1)])
    print("U is: ", U)
    return U

$$
\overline{W}_i = \frac{\overline{S}}{(1-U_i)(1-U_{i-1})}
$$

In [7]:
def cal_W_priority_nonpreemptive_MM1(lambdas, mus):
    U = cal_U(lambdas, mus)
    X_2 = cal_X2_exponential(mus)
    S = cal_S(lambdas, X_2)
    W = S / ((1 - U[1:]) * (1 - U[:-1]))
    print("W is: ", W)
    return W

def cal_R_priority_nonpreemptive_MM1(lambdas, mus):
    R = cal_W_priority_nonpreemptive_MM1(lambdas, mus) + 1 / mus
    print("R is: ", R)
    return R

$$
\begin{split}
\overline{W} &= \sum_{k=1}^p \frac{\lambda_k}{\lambda} \overline{W}_k\\
\overline{R} &= \sum_{k=1}^p \frac{\lambda_k}{\lambda} (\overline{W}_k + \overline{X}_k)\\
\end{split}
$$

In [8]:
def cal_weight(lambdas):
    lambda_ = np.sum(lambdas)
    weight = lambdas / lambda_
    print("weights are :", weight)
    return weight

def cal_total_W_priority_nonpreemptive_MM1(lambdas, mus):
    weight = cal_weight(lambdas)
    W = cal_W_priority_nonpreemptive_MM1(lambdas, mus)
    return np.sum(weight * W)

assert abs(cal_total_W_priority_nonpreemptive_MM1(np.array([2, 11]),np.array([60/11.4, 60/1.9])) * 13 - 5.704) < 0.001

def cal_total_R_priority_nonpreemptive_MM1(lambdas, mus):
    weight = cal_weight(lambdas)
    R = cal_R_priority_nonpreemptive_MM1(lambdas, mus)
    return np.sum(weight * R)

assert abs(cal_total_R_priority_nonpreemptive_MM1(np.array([0.8, 0.2]), np.array([1/0.5,1])) - 1.466) < 0.001
assert abs(cal_total_R_priority_nonpreemptive_MM1(np.array([2, 11]), np.array([60/11.4,60/1.9])) * 60 - 29.688) < 0.001

weights are : [0.15384615 0.84615385]
p is:  [0.38       0.34833333]
U is:  [0.         0.38       0.72833333]
X^2 is:  [0.0722     0.00200556]
S is:  0.08323055555555557
W is:  [0.13424283 0.49414539]
weights are : [0.8 0.2]
p is:  [0.4 0.2]
U is:  [0.  0.4 0.6]
X^2 is:  [0.5 2. ]
S is:  0.4
W is:  [0.66666667 1.66666667]
R is:  [1.16666667 2.66666667]
weights are : [0.15384615 0.84615385]
p is:  [0.38       0.34833333]
U is:  [0.         0.38       0.72833333]
X^2 is:  [0.0722     0.00200556]
S is:  0.08323055555555557
W is:  [0.13424283 0.49414539]
R is:  [0.32424283 0.52581206]


# Sistema con prioridades explusivas

$$
\begin{split}
\overline{R}_i = H_i + P_i + \overline{X}_i\\
\end{split}
$$

$\overline{H}_i$ tiempo medio en el sistema de los trabajos de las clases 1 a i que ya están en el sistema

$$
\overline{H}_i = \frac{1}{2} \frac{\sum_{k=1}^{i}\lambda_k \overline{X^2_k}}{1 - U_i}
$$

$P_i$ tiempo medio de retraso que se produce por la ejecución de todos los trabajos de las clases 1 a $i - 1$ que lleguen después que dicho trabajo y antes de que acabe su ejecución.

$$
P_i = \sum_{k=1}^{i-1}\lambda_k \overline{X}_k \overline{R}_i = \overline{R}_i U_{i-1}
$$

$\overline{X}_i$ tiempo medio de ejecución de dicho trabajo

$$
\overline{R}_i = \frac{\overline{X}_i}{1 - U_{i-1}} + \frac{\frac{1}{2}\sum_{k=1}^i \lambda_k \overline{X^2_k}}{(1 - U_i)(1 - U_{i-1})}
$$

In [9]:
def cal_R_MM1_preemptive(lambdas, mus):
    U = cal_U(lambdas, mus)
    X_2 = cal_X2_exponential(mus)
    left = (1 / mus) / (1 - U[:-1])
    print("left is: ", left)
    numerator = np.cumsum(lambdas*X_2 / 2)
    print("numerator is: ", numerator)
    denominator = (1 - U[1:]) * (1 - U[:-1])
    print("denominator is: ", denominator)
    right = numerator / denominator
    print("right is: ", denominator)
    R = np.array(left + right)
    print("R is: ", R)
    return R

$$
\overline{R} = \sum_{k=1}^{p} \frac{\lambda_k}{\lambda} \overline{R}_k
$$

In [10]:
def cal_R_total_MM1_preemptive(lambdas, mus):
    weight = cal_weight(lambdas)
    R = cal_R_MM1_preemptive(lambdas, mus)
    return np.sum(weight * R)
    
assert abs(cal_R_total_MM1_preemptive(np.array([0.5, 3, 0.2]), 1 / np.array([0.4, 0.1, 1.5])) - 0.86) < 0.1

weights are : [0.13513514 0.81081081 0.05405405]
p is:  [0.2 0.3 0.3]
U is:  [0.  0.2 0.5 0.8]
X^2 is:  [0.32 0.02 4.5 ]
left is:  [0.4   0.125 3.   ]
numerator is:  [0.08 0.11 0.56]
denominator is:  [0.8 0.4 0.1]
right is:  [0.8 0.4 0.1]
R is:  [0.5 0.4 8.6]


# Sistemas de eventos puntuales

## Distribución de N

La probabilidad de obtener k llegadas en el intervalo según una distribución con la media de llegaads por unidad de tiempo $\lambda$

$$
P(N=k) = \frac{(\lambda t)^k e^{-\lambda t}}{k!}
$$

## Distribución de T

La probabilidad de que se produzca una llegada en T, el tiempo entre dos eventos consecutivas según una distribución exponencial

$$
\begin{split}
F_T(t) &= P(T \le t) = 1 - e^{- \lambda t}\\
f_T(t) &= \lambda e^{-\lambda t}\\
\end{split}
$$

# Unión de procesos de Poisson

El número medio de llegadas será la suma de las medias de llegadas de cada uno de los procesos originales.

# División de procesos de Poisson

La división de un proceso en n secuencias, para ello, una llegada se asigna al proceso $p_k$, la tasa de llegada resulta $p_k \lambda$

# Procesos de Nacimiento y Muerte

![Diagrama de tasa de transición](diagrama-de-tasa-de-transicion.jpg)

La probabilidad de que en el sistema haya k clientes $p_k$

Número medio de clientes en el sistema $\sum_{k=0}^{\infty} kp_k=J$

Probabilidad de que haya más de r clientes $\sum_{k=r}^{\infty} p_k$

Número medio de clientes en cola (m servidores) $\sum_{k=m+1}^{\infty} p_k(k-m)$

Probabilidad de que el sistema esté ocioso $p_0$

$$
\begin{split}
p_{k-1}\lambda_{k-1} + p_{k+1}\mu_{k+1} &= p_k\lambda_k + p_k\mu_k\\
p_{i-1}\lambda_{i-1} &= p_i\lambda{i}
\end{split}
$$

$$
\begin{split}
r_i &= \frac{\lambda_{i - 1}}{\mu_i}\\
S_k &= r_1 * ... * r_k\\
p_k &= p_0S_k\\
p_0 &= \frac{1}{\sum{S_k}}
\end{split}
$$

# Cola M/M/1

$$p_k=(1-\rho)\rho^k$$

In [11]:
def cal_pk_MM1(lambdas, mus, k):
    p = cal_p(lambdas, mus)
    pk = (1 - p)*p**k
    print("p_", k, " is: ", pk)
    return pk

assert abs(cal_pk_MM1(250, 9600/1000*60, 2) - 0.1052) < 0.01

p is:  0.4340277777777778
p_ 2  is:  0.10661791054473166


$$
\overline{J} = \frac{\rho}{1 - \rho}
$$

In [12]:
def cal_J_MM1(lambdas, mus):
    p = cal_p(lambdas, mus)
    J = p / (1 - p)
    print("J is: ", J)
    return J

$$
\overline{R} = \frac{1}{\mu (1 - \rho)}
$$

In [13]:
def cal_R_MM1(lambda_, mu):
    p = cal_p(lambda_, mu)
    return 1 / (mu * (1 - p))

assert abs(cal_R_MM1(8, 10) - 0.5) < 0.001
assert abs(cal_R_MM1(8, 10) - cal_J_MM1(8, 10) / 8) < 0.001

p is:  0.8
p is:  0.8
p is:  0.8
J is:  4.000000000000001


# Sistema M/M/1 con buffer limitado

$$
p_k = \rho^k \frac{1 - \rho}{1 - \rho^{m + 1}}
$$

In [14]:
def cal_pk_MM1m(lambda_, mu, m, k):
    p = cal_p(lambda_, mu)
    pk = p**k*(1-p)/(1-p**(m+1))
    print("p_", k, " is:", pk)
    return pk

assert abs(cal_pk_MM1m(85/60, 60/37, 4, 4) * 100 - 14.988851) < 0.001

p is:  0.8736111111111111
p_ 4  is: 0.14988851075237297


$$
\overline{J}(m) = \frac{\rho}{1 - \rho} - (m + 1)\frac{\rho^{m + 1}}{1 - \rho^{m + 1}}
$$

In [15]:
def cal_J_MM1m(lambda_, mu, m):
    p = cal_p(lambda_, mu)
    J = p / (1 - p)-(m + 1)*(p**(m+1))/(1-p**(m+1))
    print("J is: ", J)
    return J

assert abs(cal_J_MM1m(1/17.2, 1/15.2, 7) - 2.8615254562386037) < 0.001

p is:  0.8837209302325582
J is:  2.8615254562386045


Fracción de tiempo que el sistema está saturado: $p_m$

Nivel medio de ocupación del buffer: $\frac{\overline{J}}{m}$

Tasa efectiva de llegadas: $\lambda (1 - p_m)$

# Sistema M/M/c (Fórmula C de Erlang)

$$
p_0 = \frac{1}{\sum_{k=0}^{c-1}\frac{(c\rho)^k}{k!} + \frac{(c\rho)^c}{c!(1 - \rho)}}
$$

In [16]:
def cal_p0_MMC_without_lost(lambdas, mus, c=1):
    c = np.atleast_1d(c)
    p = cal_p(lambdas, mus, c)
    ks = [np.arange(cc) for cc in c]
    cp = c*p
    right = (c*p)**c/(np.factorial(c)*(1-p))
    left = []
    for k, cp in zip(ks, cp):
        numerator = cp**k
        denominator = np.factorial(k)
        left.append(np.sum(numerator/denominator))
    left = np.array(left)
    p0 = 1 / (left + right)
    print("p0 is: ", p0)
    return p0

assert abs(cal_p0_MMC_without_lost(20, 10, 3) - 1/9) < 0.001

p is:  [0.66666667]
p0 is:  [0.11111111]


$$
p_k = 
     \begin{cases}
       \frac{(c\rho)^k p_0}{k!} &\quad \text{si}\, 0 \le k \le c\\
       \frac{c^c\rho^k p_0}{c!} &\quad \text{si}\, c \le k\\
     \end{cases}
$$

In [17]:
def cal_pk_MMC_without_lost(lambdas, mus, c, k):
    p = cal_p(lambdas, mus, c)
    p0 = cal_p0_MMC_without_lost(lambdas, mus, c)
    if k > 0:
        pk = p0 * c**np.minimum(c, [k]) * p**k / np.minimum(c, [k])
    else:
        pk = p0
    print("p_", k, " is: ", pk)
    return pk

assert (cal_pk_MMC_without_lost(20/60, 1/6, 3, 0) - 1/9) < 0.001
assert (cal_pk_MMC_without_lost(20/60, 1/6, 3, 1) - 2/9) < 0.001
assert (cal_pk_MMC_without_lost(20/60, 1/6, 3, 2) - 4/18) < 0.001

p is:  0.6666666666666666
p is:  [0.66666667]
p0 is:  [0.11111111]
p_ 0  is:  [0.11111111]
p is:  0.6666666666666666
p is:  [0.66666667]
p0 is:  [0.11111111]
p_ 1  is:  [0.22222222]
p is:  0.6666666666666666
p is:  [0.66666667]
p0 is:  [0.11111111]
p_ 2  is:  [0.22222222]


Número medio de procesadores ocupados $c\rho$

Probabilidad de que un trabajo tenga que hacer cola (Fórmula C de Erlang) $\sum_{k=c}^{\infty} p_k$

In [18]:
def cal_pk_serie_without_lost(lambda_, mu, c, k):
    p = cal_p(lambda_, mu, c)
    p0 = cal_p0_MMC_without_lost(lambda_, mu, c)
    p_serie = [p0]
    for i in range(1, k+1):
        p_serie.append(p_serie[i-1] * c * p / i)
    print("The p serie is ", p_serie)
    return np.array(p_serie)

In [19]:
def cal_wait_probability_MMC_without_lost(lambda_, mu, c):
    p_serie = cal_pk_serie_without_lost(lambda_, mu, c, c)
    return 1 - np.sum(p_serie[:-1])

assert abs(cal_wait_probability_MMC_without_lost(20, 10, 3) - 4/9) < 0.001
assert abs(cal_wait_probability_MMC_without_lost(16 / 60, 1 / 2.3, 1) * 100 - 61.33) < 0.01

p is:  0.6666666666666666
p is:  [0.66666667]
p0 is:  [0.11111111]
The p serie is  [array([0.11111111]), array([0.22222222]), array([0.22222222]), array([0.14814815])]
p is:  0.6133333333333333
p is:  [0.61333333]
p0 is:  [0.38666667]
The p serie is  [array([0.38666667]), array([0.23715556])]


Número medio de trabajos en cola

$$
\overline{Q} = \frac{(c\rho)^c \lambda \mu p_0}{(c-1)(c \mu + \lambda)^2}
$$

In [20]:
def cal_Q_MMC(lambda_, mu, c):
    p = cal_p(lambda_, mu, c)
    p0 = cal_p0_MMC_without_lost(lambda_, mu, c)
    Q = ((c*p)**c * lambda_ * mu * p0) / (np.factorial(c-1)*(c*mu-lambda_)**2)
    print("Q is: ", Q)
    return Q

def cal_W_MMC(lambda_, mu, c):
    W = cal_Q_MMC(lambda_, mu, c) / lambda_
    print("W is: ", W)
    return W

def cal_R_MMC(lambda_, mu, c):
    return cal_W_MMC(lambda_, mu, c) + 1 / mu

assert abs(cal_R_MMC(16 / 60, 1 / 2.3, 1) - 5.948) < 0.001

p is:  0.6133333333333333
p is:  [0.61333333]
p0 is:  [0.38666667]
Q is:  [0.97287356]
W is:  [3.64827586]


# Sistema M/M/c con pérdidas

Si $\rho$ es $\frac{\lambda}{\mu}$

$$p_k = \frac{\rho^k}{k!}\frac{1}{\sum_{j=0}^c\frac{p^j}{j!}}$$

$p_c$ es la probabilidad de que el sistema está lleno, Fórmula B de Erlang

# Sistema M/M/c con buffer limitado

$$
\begin{split}
p_0
&= \frac{1}{\sum_{k=0}^{c-1}\frac{(c\rho)^k}{k!} + \sum_{k=c}^{m}\frac{c^c\rho^k}{c!}}\\
&= \frac{1}{\sum_{k=0}^{c-1}\frac{(c\rho)^k}{k!} + \frac{(1-\rho^{m+1-c})\lambda^c}{c! \mu^c (1-\rho)}}\\
\end{split}
$$

In [21]:
def cal_p0_MMcm(lambda_, mu, c, m):
    p = cal_p(lambda_, mu, c)
    p0 = 1 / (np.sum([(c*p)**k/factorial(k) for k in range(c)]) + (1-p**(m+1-c))*lambda_**c/(factorial(c)*mu**c*(1-p)))
    print("p0 is: ", p0)
    return p0

assert abs(cal_p0_MMcm(20, 10, 3, 5) - 0.128) < 0.001

p is:  0.6666666666666666
p0 is:  0.12796208530805686


$$
p_k = 
     \begin{cases}
       \frac{(c\rho)^k p_0}{k!} &\quad \text{si}\, 0 \le k \le c\\
       \frac{c^c\rho^k p_0}{c!} &\quad \text{si}\, c \le k \le m\\
     \end{cases}
$$

In [22]:
def cal_pk_MMcm(lambda_, mu, c, m, k):
    p = cal_p(lambda_, mu, c)
    p0 = cal_p0_MMcm(lambda_, mu, c, m)
    pk = (c*p)**k/factorial(k)*p0 if k < c else c**c*p**k/factorial(c)*p0
    print("p_", k, " is: ", pk)
    return pk

assert abs(cal_pk_MMcm(20, 10, 3, 5, 5) - 0.0758) < 0.01
assert cal_pk_MM1m(12/60, 1/4, 1, 0) - cal_pk_MMcm(12/60, 1/4, 1, 1, 0) < 0.001

p is:  0.6666666666666666
p is:  0.6666666666666666
p0 is:  0.12796208530805686
p_ 5  is:  0.0758293838862559
p is:  0.8
p_ 0  is: 0.5555555555555556
p is:  0.8
p is:  0.8
p0 is:  0.5555555555555556
p_ 0  is:  0.5555555555555556


$$\overline{J} = \sum_{k=0}^{m} k p_k$$

In [23]:
def cal_J_MMcm(lambda_, mu, c, m):
    p = cal_p(lambda_, mu, c)
    p0 = cal_p0_MMcm(lambda_, mu, c, m)
    p_serie = np.array([c*p/max(k,1) if k < c else p for k in range(m+1)])
    p_serie[0] = p0
    p_serie = np.cumprod(p_serie)
    print("The p serie is ", p_serie)
    J = np.sum(p_serie * np.arange(p_serie.size))
    print("J is: ", J)
    return J

assert abs(cal_J_MMcm(20, 10, 3, 5) - 2.1137) < 0.01
assert abs(cal_J_MMcm(20, 10, 1, 5) - cal_J_MM1m(20, 10, 5)) < 0.000001

p is:  0.6666666666666666
p is:  0.6666666666666666
p0 is:  0.12796208530805686
The p serie is  [0.12796209 0.25592417 0.25592417 0.17061611 0.11374408 0.07582938]
J is:  2.1137440758293837
p is:  2.0
p is:  2.0
p0 is:  0.015873015873015872
The p serie is  [0.01587302 0.03174603 0.06349206 0.12698413 0.25396825 0.50793651]
J is:  4.095238095238095
p is:  2.0
J is:  4.095238095238095


In [24]:
def cal_R_MMcm(lambda_, mu, c, m):
    J = cal_J_MMcm(lambda_, mu, c, m)
    R = J / lambda_
    print("R is: ", R)
    return R

assert abs(cal_R_MMcm(20, 10, 3, 5) - 0.1056) < 0.01

p is:  0.6666666666666666
p is:  0.6666666666666666
p0 is:  0.12796208530805686
The p serie is  [0.12796209 0.25592417 0.25592417 0.17061611 0.11374408 0.07582938]
J is:  2.1137440758293837
R is:  0.10568720379146919


In [25]:
def cal_W_MMcm(lambda_, mu, c, m):
    R = cal_R_MMcm(lambda_, mu, c, m)
    W = R - 1/mu
    print("W is: ", W)
    return W

assert abs(cal_W_MMcm(20, 10, 3, 5) * 60 - 0.34) < 0.01

p is:  0.6666666666666666
p is:  0.6666666666666666
p0 is:  0.12796208530805686
The p serie is  [0.12796209 0.25592417 0.25592417 0.17061611 0.11374408 0.07582938]
J is:  2.1137440758293837
R is:  0.10568720379146919
W is:  0.005687203791469184


# Redes de colas

Si una **cola M/M/m estable** en la que los procesos de llegada y servicio son **procesos de Poison** con tasas $\lambda$ y $\mu$, el prceso de salida es un proceso de Poisson de tasa $\lambda$

Ses $q_{ij}$ la probabilidad de que un trabajo llega al nodo $i$ y después de haber terminado el servicio requerido en este nodo, pasa al nodo $j$, y sale de la red con probabilidad $1-\sum_{j=1}^{n} q_{ij}$

![Teorema de Jackson](teorema-de-jackson.jpg)

$$
\overline{R}_i = \frac{1}{\mu_i - I_i}
$$

In [26]:
def cal_R_queueing_netowork_MM1(I, mu):
    R = 1 / (mu - I)
    print("R is: ", R)
    return R

$$
\overline{R} = \frac{\sum_i I_i \overline{R}_i}{\sum_k \lambda_k}
$$

In [27]:
def cal_R_total_queueing_network_MM1(I, mu, lambdas):
    lambda_ = np.sum(lambdas)
    R = cal_R_queueing_netowork_MM1(I, mu)
    return np.sum(I / lambda_ * R)

assert abs(cal_R_total_queueing_network_MM1(np.array([1000, 550, 285.5, 872.75, 549.75]), np.array([1357, 727, 375, 1192, 666]), 1550) * 60 - 0.641) < 0.001

R is:  [0.00280112 0.00564972 0.01117318 0.00313234 0.00860215]


# Autoevaluación

## Problema 1

Un sistema de comunicaciones recibe mensajes con una tasa de 8 trabajos por segundo. Se sabe que tanto la llegada como el servicio son procesos de Poisson y que el sistema tiene un factor de utilización del 80%

### M/M/1 $\in$ M/G/1

In [28]:
lambda_ = 8

In [29]:
p = 0.8

In [30]:
X = p / lambda_

In [31]:
mu = 1 / X

In [32]:
X_square = cal_X2_exponential(mu)

X^2 is:  0.02


Fórmula de Pollaxe-Khintchine

¿Cuál es el tiempo medio de espera?

In [33]:
W = cal_W_MG1(lambda_, mu, CX=1)
W

p is:  0.8
W is:  0.40000000000000013


0.40000000000000013

### ¿Cuál es el tiempo medio de respuesta? Respuesta

In [34]:
R = W + X
R

0.5000000000000001

### ¿Cuál es el número medio de mensajes en cola?

In [35]:
Q = lambda_ * W
Q

3.200000000000001

### ¿Cuál es el número medio de mensajes en el sistema? Respuesta

In [36]:
J = lambda_ * R
J

4.000000000000001

## Problema 2

Un centro de atención al cliente recibe llamadas de dos tipos: contratación de servicios y petición de información. Las llamadas se producen según procesos de Poisson con tasas de 2 y 11 llamadas por hora según sea de contratación o de petición de información, respectivamente. Las llamadas de contratación requieren una media de 11.4 minutos, mientras que las de petición de información se atienden en una media de 1.9 minutos. Actualmente no hay discriminaciónn en el servicio, pero se ha detectado que algunos clientes que llaman para contratar un servicio abandonan la llamada antes de ser atendidos. Por ello, se ha decidido que se va a implantar un sistema que dé prioridad a estas llamadas. Analizar el modelo que quedaría después de implantar el sistema de prioridades no expulsivas.

In [37]:
lambdas = np.array([2, 11])
mus = 1 / (np.array([11.4, 1.9]) / 60)
print(lambdas, mus)

[ 2 11] [ 5.26315789 31.57894737]


### El sistema actual (sin prioridades) se ajusta a un modelo M/M/1

### ¿Cuál es el tiempo medio de espera de las llamadas de contratación de servicios?

Sistema no expulsivos con distintas clases de prioridad

In [38]:
W = cal_W_priority_nonpreemptive_MM1(lambdas, mus)

p is:  [0.38       0.34833333]
U is:  [0.         0.38       0.72833333]
X^2 is:  [0.0722     0.00200556]
S is:  0.08323055555555553
W is:  [0.13424283 0.49414539]


### ¿Cuál es el tiempo medio de espera de las llamadas de contratación de servicios? Respuesta

In [39]:
W[0] * 60

8.054569892473115

¿Cuál es el tiempo medio de espera de las llamadas de solicitud de información? Respuesta

In [40]:
W[1] * 60

29.648723530575875

¿Cuál es el número medio de llamadas en cola? Respuesta

In [41]:
Q = np.sum(lambdas * W)
Q

5.704084977021347

### ¿Cuál es el tiempo medio de respuesta (tiempo que transcurre desde que se inicia hasta que finaliza la llamada)?

In [42]:
cal_total_R_priority_nonpreemptive_MM1(lambdas, mus) * 60

weights are : [0.15384615 0.84615385]
p is:  [0.38       0.34833333]
U is:  [0.         0.38       0.72833333]
X^2 is:  [0.0722     0.00200556]
S is:  0.08323055555555553
W is:  [0.13424283 0.49414539]
R is:  [0.32424283 0.52581206]


29.688084509329297

## Problema 3

Una centralita de venta de entradas por teléfono está preparada atender 2 llamadas simultáneamente sin capacidad para mantener llamadas en espera, de forma que si se recibe una llamada y los 2 operadores están ocupados, la llamada se pierde. Las llamadas se producen según un proceso de Poisson con una tasa de 85 llamadas a la hora, con una duración media de 59 segundos (el proceso de servicio también se ajusta a un proceso de Poisson)

In [43]:
c = 2
lambda_ = 85 / 60
X = 59 / 60
mu = 1 / X

### Este sistema se ajusta a un modelo M/M/c con perdida

### ¿Qué porcentaje de tiempo la centralita llena?

In [44]:
p_c = cal_pk_MMcm(lambda_, mu, c, c, c)
p_c * 100

p is:  0.6965277777777779
p is:  0.6965277777777779
p0 is:  0.29732195166855024
p_ 2  is:  0.2884920517709555


28.84920517709555

¿Qué porcentaje de llamadas se pierden?

In [45]:
p_c * 100

28.84920517709555

La empresa se queda sólo con el operador más eficiente, que en promedio atiende las llamadas en sólo 37 segundos, y contrata un sistema de servicio en espera con capacidad para 3 llamadas, de forma que cuando se recibe una llamada, si el operador está ocupado y ya hay 3 llamadas en espera, la centralita emite la señal de línea ocupada, la llamada se pierde y el cliente se va a la competencia

### Este sistema se ajusta a un modelo M/M/1 con buffer limitado

### ¿Qué porcentaje de llamadas se pierden?

In [46]:
X = 37 / 60
mu = 1 / X
m = 3 + 1

In [47]:
p_m = cal_pk_MM1m(lambda_, mu, m, m)
p_m * 100

p is:  0.8736111111111112
p_ 4  is: 0.14988851075237306


14.988851075237305

### En esta nueva situación, ¿cuál es el la tasa efectiva de llegada?

In [48]:
lambda_ * (1 - p_m)

1.2043246097674716

### ¿Cuál es el nivel medio de ocupación del buffer?

In [49]:
cal_J_MM1m(lambda_, mu, m) / m

p is:  0.8736111111111112
J is:  1.731875095426231


0.43296877385655774

## Problema 4

Una ventanilla de un banco realiza operaciones en un tiempo medio de 2.3 minutos y los clientes llegan con una tasa media de 16 clientes a la hora. Si se supone que las llegadas siguen un proceso de Poisson y el tiempo de servicio es exponencial.

### Este sistema se ajusta a un modelo Respuesta M/M/1

In [50]:
X = 2.3
lambda_ = 16 / 60
mu = 1 / lambda_
print(lambda_, X)

0.26666666666666666 2.3


### ¿Qué porcentaje de tiempo el cajero está ocioso? Respuesta

In [51]:
p = lambda_* X

In [52]:
p0 = (1 - p)
p0 * 100

38.66666666666667

### ¿Qué porcentaje de clientes deben esperar en cola?

In [53]:
(1 - p0) * 100

61.33333333333333

### ¿Cuál es el tiempo medio de estancia de los clientes en el banco

In [54]:
J = p / (1 - p)
J

1.5862068965517238

### ¿Cuál es el tiempo medio de estancia de los clientes en el banco?

In [55]:
R = J / lambda_
R

5.948275862068964

In [56]:
W = R - X
W

3.6482758620689646

## Problema 5

Una estación de ITV ha sufrido una avería y sólo dispone un equipo de inspección con espacio para 6 coches más en espera. El proceso de llegadas se puede considerar como un proceso de Poisson con una tasa media de un coche cada 17.2 minutos. El tiempo de servicio se puede aproximar mediante una variable aleatoria con distribución exponencial de media 15.2 minutos. Si un coche llega a la estación y esta está saturada, se marcha sin esperar a que quede una plaza libre.

### Este sistema se ajusta a un modelo M/M/1 con buffer limitado

In [57]:
m = 6 + 1
lambda_ = 1 / 17.2
X = 15.2
mu = 1 / X
print(lambda_, mu, m)

0.05813953488372093 0.06578947368421052 7


### ¿Cuál es la probabilidad de que un coche llegue y se encuentre todos los puestos ocupados?

In [58]:
p_m = cal_pk_MM1m(lambda_, mu, m, m)
p_m

p is:  0.8837209302325582
p_ 7  is: 0.07793543657502297


0.07793543657502297

### ¿Cuál es la tasa efectiva de llegada?

In [59]:
tasa_efectiva = lambda_ * (1 - p_m) * 60
tasa_efectiva

3.2165042910173613

### ¿Cuál es el número medio de coches en el sistema?

In [60]:
cal_J_MM1m(lambda_, mu, m)

p is:  0.8837209302325582
J is:  2.8615254562386045


2.8615254562386045

La empresa ha calculado el beneficio por vehículo en 22 euros. Tiene la posibilidad de ampliar la capacidad de la zona de espera en un vehículo más, con lo que la proporción de tiempo en que la estación está saturada disminuye y puede atender a más vehículos Suponiendo que está abierta 8 horas al día, durante 22 días al mes y que el proceso de llegadas es el mismo durante todo el horario de apertura, ¿cuánto sería el incremento mensual en el beneficio esperado por poder atender a los nuevos coches?

In [61]:
m = 6 + 2

In [62]:
precio = 22

In [63]:
hours = 8

In [64]:
days = 22

In [65]:
p_m = cal_pk_MM1m(lambda_, mu, m, m)
p_m

p is:  0.8837209302325582
p_ 8  is: 0.06443531189842151


0.06443531189842151

In [66]:
(lambda_ * (1 - p_m) * 60 - tasa_efectiva) * hours * days * precio

182.34587005046873

## Problema 6

Un call center ha recogido datos de las llamadas telefónicas recibidas en el último periodo durante la hora punta. El número de llamadas recibidas se puede aproximar mediante una variable aleatoria con distribución Poisson de media 51 llamadas a la hora. El análisis de las conversaciones revela que la duración media de las llamadas es de 2.5 minutos. Actualmente tiene 3 personas atendiendo el teléfono. Si cuando una persona llama, todos los operarios están ocupados, la llamada se pierde.

### Este sistema se ajusta a un modelo M/M/m con pérdidas

In [67]:
lambda_ = 51 / 60
X = 2.5
c = 3
mu = 1 / X
print(lambda_, mu, c)

0.85 0.4 3


### ¿Cuál es la probabilidad de que la llamada se pierda? 

In [68]:
p_c = cal_pk_MMcm(lambda_, mu, c, c, c)
p_c

p is:  0.7083333333333334
p is:  0.7083333333333334
p0 is:  0.14322346030117955
p_ 3  is:  0.22905496759755706


0.22905496759755706

### ¿Qué porcentaje de tiempo están todos los operarios ocupados?

In [69]:
p_c * 100

22.905496759755707

### ¿Qué porcentaje de tiempo están todos los operarios libres?

In [70]:
p_empty = cal_p0_MMcm(lambda_, mu, c, c)
p_empty * 100

p is:  0.7083333333333334
p0 is:  0.14322346030117955


14.322346030117956

La empresa quiere aumentar el nivel de servicio, reduciendo la probabilidad de que una llamada se encuentre a todos los operarios ocupados. ¿Cuál es el número mínimo de operarios que garantiza que esta probabilidad es inferior a 0.10? Respuesta operarios

In [71]:
k = c
p_k = p**k/factorial(k)/sum([p**j/factorial(j) for j in range(0, k+1)])
while p_k > 0.1:
    k += 1
    p_k = p**k/factorial(k)/sum([p**j/factorial(j) for j in range(0, k+1)])
k

3

## Problema 7

En un determinado se ha implantado un nuevo sistema para el control de seguridad de acceso de los viajeros al área de embarque. Se ha dispuesto una puerta que da acceso a dos arcos detectores de metales que deben superar todos los viajeros de la cola. En caso de que el arco detector no señale ninguna incidencia, el viajero pasa directamente a la zona de embarque. En caso de que detecte algo extraño, el viajero debe pasar a un escáner corporal 3D. Este escaner es el mismo para las dos puertas, de forma que hay una única cola con los viajeros precedentes de las dos puertas. En el caso en el que el escaner corporal 3D siga detectando algo extraño, el viajero debe pasar a un tercer control más exhaustivo, para lo que pasa a unas dependencias especiales del aeropuerto.

Una vez en los viajeros traspasan la puerta de acceso, se reparten por igual entre los dos arcos de seguridad. El 83% de los viajeros que acceden a la zona de embarque, pasan sin problemas por primer arco detector, independientemente de la puerta que usen. El 11% de los viajeros que pasan por el escaner corporal 3D necesitan pasar por las dependencias especiales, mientras que el resto (89%) pasan directamente a la sala de embarque.

Por tanto, se puede considerar que el sistema tiene cinco servidores:

S1 : puerta de acceso a la que llegan todos los viajeros con una tasa λ=80 viajeros por minuto. Este servidor atiende a los viajeros con una tasa μ1 =107 viajeros por minuto.

S2 : primer arco que recibe a la mitad de los viajeros de la puerta de acceso. Este arco tiene una tasa de inspección de μ2 =52 viajeros por minuto.

S3 : segundo arco que recibe a la mitad de los viajeros de la puerta de acceso. Este arco tiene una tasa de inspección de μ3 =52 viajeros por minuto.

S4 : escaner corporal 3D, que recibe a los viajeros a los que los arcos anteriores detectan algo extraño. Este escaner trabaja con una tasa de μ4 =18 viajeros por minuto.

S5 : dependencias especiales, que recibe a todos los viajeros a los que el escaner visual 3D detecta algo extraño. Una vez que ha pasado por aquí, un viajero pasa a la zona de embarque. Cada viajero invierte un promedio de x5 =30 segundos en pasar este proceso.

![Diseño de redes](problema7.png)

Denotamos por Ii la tasa de llegada al servidor i.
Nota: a la hora de realizar los cálculos, es necesario tener en cuenta las unidades eque se utilizan para cada parámetro del problema y las unidades en las que se piden los resultados.
Completa adecuadamente las ecuaciones de tráfico:

In [72]:
lambda_ = 80
mu = np.array([107, 52, 52, 18, 60/30])

In [73]:
q12 = 0.5
q13 = 0.5
q24 = 0.17
q34 = 0.17
q45 = 0.11

In [74]:
I1 = lambda_
I2 = q12 * I1
I3 = q13 * I1
I4 = q24 * I2 + q34 * I3
I5 = q45 * I4
I = np.array([I1, I2, I3, I4, I5])
print(I)

[80.    40.    40.    13.6    1.496]


El número medio de viajeros en cada servidor Ji es:

In [75]:
J = I/(mu-I)
J

array([2.96296296, 3.33333333, 3.33333333, 3.09090909, 2.96825397])

Y el número medio de viajeros en todo el sistema J es Respuesta

In [76]:
np.sum(J)

15.688792688792692

## Problema 8

Una empresa recibe documentos de dos tipos, 1 y 2. Los documentos de tipo 1 deben pasar por el OCR número 1, mientras que los de tipo 2 pasan por el OCR 2. Como resultado de la lectura por el correspondiente OCR, cada documento puede: a) ser rechazado (si la lectura presentó ligeros errores) y sale del sistema, b) pasar a grabación manual (si la lectura fue imposible) o c) pasar directamente al proceso de almacenamiento (distinto para cada tipo de documento). Si un documento pasa por grabación manual, después pasa a cualquiera de los dos procesos de almacenamiento anteriores con la misma probabilidad. Se ha detectado que el 10% de los documentos de tipo 1 son rechazados, el 73% salen correctamente y el 17% restante deben pasar a grabación manual. En el OCR2 (documentos de tipo 2), estos porcentajes son 5%, 74% y 21%, respectivamente.

Por tanto, se puede considerar que el sistema tiene cinco servidores:

S1 : primer OCR, OCR1, que recibe documentos de tipo 1 sólo del exterior con una tasa λ1 =1000 documentos por hora. Los documentos rechazados en este punto, salen del sistema. Este sistema procesa documentos con una tasa μ1 =1357 documentos por hora.

S2 : segundo OCR, OCR2, que recibe documentos de tipo 2 sólo del exterior con una tasa λ2 =550. Los documentos rechazados en este punto, salen del sistema. Este sistema procesa documentos con una tasa μ2 =727 documentos por hora.

S3 : grabación manual, que recibe los documentos de tipo 1 y 2 cuya lectura ha resultado defectuosa. Este sistema procesa documentos con una tasa μ3 =375 documentos por hora.

S4 : primer almacenamiento, que recibe los documentos de tipo 1 que han sido leídos correctamente en el OCR1 y la mitad de los documentos que han pasado por la grabación manual. Una vez que ha pasado por aquí, un documento sale del sistema. Este sistema procesa documentos con una tasa μ4 =1192 documentos por hora.

S5 : primer almacenamiento, que recibe los documentos de tipo 2 que han sido leídos correctamente en el OCR1 y la mitad de los documentos que han pasado por la grabación manual. Una vez que ha pasado por aquí, un documento sale del sistema. Este sistema procesa documentos con una tasa μ5 =666 documentos por hora.

![Diseño de redes](problema8.png)

In [77]:
lambda1 = 1000
lambda2 = 550
mu = np.array([1357, 727, 375, 1192, 666])
q13 = 0.17
q23 = 0.21
q14 = 0.73
q25 = 0.74
q34 = 0.5
q35 = 0.5

Denotamos por Ii la tasa de llegada al servidor i.

Nota: a la hora de realizar los cálculos, es necesario tener en cuenta las unidades eque se utilizan para cada parámetro del problema y las unidades en las que se piden los resultados.

### Completa adecuadamente las ecuaciones de tráfico:

In [78]:
I1 = lambda1
I2 = lambda2
I3 = q13 * I1 + q23 * I2
I4 = q14 * I1 + q34 * I3
I5 = q25 * I2 + q35 * I3
I = np.array([I1, I2, I3, I4, I5])
print(I)

[1000.    550.    285.5   872.75  549.75]


### El tiempo medio de respuesta de cada servidor es:

In [79]:
R = 1/(mu-I)
R * 60

array([0.16806723, 0.33898305, 0.67039106, 0.18794049, 0.51612903])

### Y el tiempo medio de respuesta global es

In [80]:
np.sum(I*R) / (lambda1 + lambda2) * 60

0.6410784174947248

# Serie de problemas y ejercicios de Teoría de Colas

## Problema 1

Una empresa dispone de dos impresoras y cada una atiende la demanda de un departamento. Ambas impresoras son iguales y trabajan a un ritmo promedio de 1.1 trabajos por minuto. El departamento A envía un promedio de 45 trabajos por hora y el departamento B un trabajo cada 90 segundos. Las impresoras tienen capacidad suficiente como para mantener en espera todos los trabajos que se reciban. Tanto las llegadas como el servicio se pueden considerar procesos de Poisson.

In [81]:
lambda1 = 45 / 60
lambda2 = 60 / 90
mu = 1.1
print(lambda1, lambda2, mu)

0.75 0.6666666666666666 1.1


### ¿A qué modelo se ajustan los sistemas descritos? **M/M/1**

### ¿Qué proporción de trabajos tienen que esperar para ser atendidos en cada departamento?

#### Departamento A

In [82]:
cal_p(lambda1, mu)

p is:  0.6818181818181818


0.6818181818181818

#### Departamento B

In [83]:
cal_p(lambda2, mu)

p is:  0.606060606060606


0.606060606060606

### ¿Cuál es el tiempo medio de respuesta en cada una de las dos alternativas?

#### Departamento A:

In [84]:
cal_R_MM1(lambda1, mu)

p is:  0.6818181818181818


2.8571428571428563

#### Departamento B:

In [85]:
cal_R_MM1(lambda2, mu)

p is:  0.606060606060606


2.307692307692307

La empresa modifica el sistema, unificando las dos colas en una única, de forma que cuando una de las dos impresoras se queda libre, pasa a imprimir el primer trabajo de la cola, independientemente del departamento que lo envió.

### ¿A qué modelo se ajustan los sistemas descritos? **M/M/c**

In [86]:
c = 2

In [87]:
lambda_ = lambda1 + lambda2
lambda_

1.4166666666666665

### ¿Cuál es el tiempo medio de respuesta de los trabajos de cada departamento en esta nueva situación?

#### Departamento A:

In [88]:
cal_R_MMC(lambda_, mu, c)

p is:  0.6439393939393938
p is:  [0.64393939]
p0 is:  [0.21658986]
Q is:  [0.91233692]
W is:  [0.64400253]


array([1.55309344])

#### Departamento B:

In [89]:
cal_R_MMC(lambda_, mu, c)

p is:  0.6439393939393938
p is:  [0.64393939]
p0 is:  [0.21658986]
Q is:  [0.91233692]
W is:  [0.64400253]


array([1.55309344])

La empresa sustituye las dos impresoras por una única impresora el doble de rápido y da más prioridad a los trabajos del departamento A. Una vez iniciada la impresión de un trabajo, no se puede interrumpir.

In [90]:
mu *= 2

### ¿Cuál es el tiempo medio de respuesta de los trabajos de cada departamento, A y B

In [91]:
lambdas = np.array([lambda1, lambda2])
mus = np.array([mu, mu])

In [92]:
R = cal_R_priority_nonpreemptive_MM1(lambdas, mus)

p is:  [0.34090909 0.3030303 ]
U is:  [0.         0.34090909 0.64393939]
X^2 is:  [0.41322314 0.41322314]
S is:  0.29269972451790627
W is:  [0.44409613 1.24724872]
R is:  [0.89864159 1.70179417]


#### Departamento A:

In [93]:
R[0]

0.8986415882967606

#### Departamento B:

In [94]:
R[1]

1.701794170612952

## Problema 2

Una oficina bancaria tiene dos cajeros separados y con un funcionamiento independiente. Cada cajero recibe el 50% de los clientes que requieren este servicio en la oficina bancaria. El banco se está planteando sustituir la organización anterior (que llamaremos alternativa 1) por un sistema con una cola única que sirva a los dos cajeros (el primer cliente de la cola sería atendido por el primer cajero que quede libre), que llamaremos alternativa 2. El banco recibe un promedio de 38 clientes por hora que requieren servicios en el cajero por un promedio de 1.5 minutos para realizar sus gestiones. Tanto las llegadas como el servicio se pueden considerar procesos de Poisson y en todos los casos se puede suponer que las colas tienen capacidad ilimitada.

In [95]:
lambda2 = 38 / 60
mu = 1 / 1.5
print(lambda2, mu)

0.6333333333333333 0.6666666666666666


In [96]:
lambda1 = lambda2 / 2
c = 2

### ¿Qué modelos se ajustan a cada una de las dos alternativas?

Alternativa 1: **M/M/1**

Alternativa 2: **M/M/c**

### ¿Cuál es la tasa de uso del sistema en cada una de las dos alternativas?

#### Alternativa 1:

In [97]:
cal_p(lambda1, mu)

p is:  0.475


0.475

#### Alternativa 2:

In [98]:
cal_p(lambda2, mu, c)

p is:  0.475


0.475

### ¿Cuál es el tiempo medio de respuesta en cada una de las dos alternativas?

#### Alternativa 1:

In [99]:
cal_R_MM1(lambda1, mu)

p is:  0.475


2.857142857142857

#### Alternativa 2:

In [100]:
cal_R_MMC(lambda2, mu, c)

p is:  0.475
p is:  [0.475]
p0 is:  [0.3559322]
Q is:  [0.2767958]
W is:  [0.437046]


array([1.937046])

### ¿Cuál es el tiempo medio de espera en cada una de las dos alternativas?

#### Alternativa 1:

In [101]:
cal_W_MG1(lambda1, mu)

p is:  0.475
W is:  1.3571428571428572


1.3571428571428572

#### Alternativa 2:

In [102]:
cal_W_MMC(lambda2, mu, c)

p is:  0.475
p is:  [0.475]
p0 is:  [0.3559322]
Q is:  [0.2767958]
W is:  [0.437046]


array([0.437046])

### ¿Cuál es la probabilidad de que un cliente tenga que un esperar en cada una de las dos alternativas?

#### Alternativa 1:

In [103]:
cal_wait_probability_MMC_without_lost(lambda1, mu, 1)

p is:  0.475
p is:  [0.475]
p0 is:  [0.525]
The p serie is  [array([0.525]), array([0.249375])]


0.475

#### Alternativa 2:

In [104]:
cal_wait_probability_MMC_without_lost(lambda2, mu, c)

p is:  0.475
p is:  [0.475]
p0 is:  [0.3559322]
The p serie is  [array([0.3559322]), array([0.33813559]), array([0.16061441])]


0.3059322033898304

Los cajeros funcionan 8 horas al día y el banco paga una penalización de 0.1 euros por cada minuto en el que en la cola hay al menos un cliente esperando. 

### ¿Cuál es el coste diario esperado de esta penalización en la alternativa 2?

In [105]:
price = 0.1

In [106]:
(1 - np.sum(cal_pk_serie_without_lost(lambda2, mu, c, c))) * price * 60 * 8

p is:  0.475
p is:  [0.475]
p0 is:  [0.3559322]
The p serie is  [array([0.3559322]), array([0.33813559]), array([0.16061441])]


6.9752542372881265

## Problema 3

Un sistema con dos procesadores recibe trabajos de dos tipos distintos, A y B, con una tasa de 46 y 56 trabajos a la hora, respectivamente. Cada trabajo, independientemente del tipo, tarda en procesarse una media de 50 segundos y es atendido por un único procesador. La cola de espera es suficientemente grande como para mantener en espera todos los trabajos que se reciban. Tanto las llegadas como el servicio se pueden considerar como un proceso de Poisson.

In [107]:
lambdaA = 46 / 60
lambdaB = 56 / 60
mu = 60 / 50
print(lambdaA, lambdaB, mu)

0.7666666666666667 0.9333333333333333 1.2


In [108]:
c = 2
lambda_ = lambdaA + lambdaB

### ¿A qué modelo se ajusta este sistema? M/M/c

### ¿Qué proporción de trabajos tienen que esperar para ser atendidos?

In [109]:
cal_wait_probability_MMC_without_lost(lambda_, mu, c) * 100

p is:  0.7083333333333335
p is:  [0.70833333]
p0 is:  [0.17073171]
The p serie is  [array([0.17073171]), array([0.24186992]), array([0.17132453])]


58.73983739837401

### ¿Cuál es el tiempo medio de espera en el sistema?

In [110]:
cal_W_MMC(lambda_, mu, c)

p is:  0.7083333333333335
p is:  [0.70833333]
p0 is:  [0.17073171]
Q is:  [1.42653891]
W is:  [0.83914053]


array([0.83914053])

Uno de los procesadores se estropea y para paliar la situación, el gestor del sistema decide duplicar la velocidad de proceso de cada trabajo y dar más prioridad a los trabajos de tipo B (aunque en ningún caso se interrumpe el proceso de un trabajo ya en ejecución).

In [111]:
mu *= 2

### ¿Cuál es el tiempo medio de espera de cada uno de los trabajos, A y B?

In [112]:
W_B, W_A = cal_W_priority_nonpreemptive_MM1(np.array([lambdaB, lambdaA]), np.array([mu]*2))

p is:  [0.38888889 0.31944444]
U is:  [0.         0.38888889 0.70833333]
X^2 is:  [0.34722222 0.34722222]
S is:  0.2951388888888889
W is:  [0.48295455 1.65584416]


#### Trabajos de tipo A:

In [113]:
W_A

1.655844155844156

#### Trabajos de tipo B:

In [114]:
W_B

0.4829545454545454

### Promedio de ambos tipos de trabajo:

In [115]:
cal_total_W_priority_nonpreemptive_MM1(np.array([lambdaB, lambdaA]), np.array([mu]*2))

weights are : [0.54901961 0.45098039]
p is:  [0.38888889 0.31944444]
U is:  [0.         0.38888889 0.70833333]
X^2 is:  [0.34722222 0.34722222]
S is:  0.2951388888888889
W is:  [0.48295455 1.65584416]


1.0119047619047619

## Problema 4

Un agente comercial recibe todas sus llamadas en el movil y, para no perder ninguna llamada, ha considerado adecuado desactivar el buzón de voz. De esta forma, cuando recibe una llamada y está ocupado, la llamada se pierde. Recibe un promedio de 12 llamadas a la hora, con una duración media de 4 minutos. Tanto la recepción de llamadas como el tiempo de llamada se pueden considerar procesos de Poisson.

In [116]:
lambda_ = 12 / 60
mu = 1/4
print(lambda_, mu)

0.2 0.25


In [117]:
m = 1

### Este sistema se ajusta a un modelo M/M/1 con buffer limitado

### ¿Que porcentaje de tiempo el agente está ocioso, esperando llamadas?

In [118]:
cal_pk_MM1m(lambda_, mu, m, 0) * 100

p is:  0.8
p_ 0  is: 0.5555555555555556


55.55555555555556

### ¿Que porcentaje de tiempo el agente está ocupado, atendiendo llamadas?

In [119]:
cal_pk_MM1m(lambda_, mu, m, 1) * 100

p is:  0.8
p_ 1  is: 0.44444444444444453


44.44444444444445

### ¿Qué porcentaje de llamadas se pierden? Respuesta

In [120]:
cal_pk_MM1m(lambda_, mu, m, 1) * 100

p is:  0.8
p_ 1  is: 0.44444444444444453


44.44444444444445

Su compañía de teléfono le ofrece un servicio de llamada en espera de capacidad ilimitada. El agente ha calculado que cada llamada atendida le reporta un beneficio de 10 euros. 

In [121]:
profit = 10

### ¿Cuál es el precio máximo por hora que tendría que estar dispuesto a pagar por el alquiler del servicio para que le resulte rentable contratarlo?

In [122]:
lambda_ * cal_pk_MM1m(lambda_, mu, m, 1) * 60 * profit

p is:  0.8
p_ 1  is: 0.44444444444444453


53.33333333333334

El sistema contratado, además, le permite clasificar los clientes y discriminar en función de esta clasificación. El agente ha detectado que el 20% de sus clientes son mejores que el resto y los ha clasificado como super cuquis ya que, a pesar de requerir el mismo tiempo de servicio, le proporcionan un beneficio medio de 30 euros. Por ello, ha decidido darles prioridad y si cuando termina una llamada, hay algún cliente super cuqui en espera, es atendido antes que el resto. 

In [123]:
p_super_cuquis = 0.2

### ¿Cuál es el tiempo medio de respuesta de los clientes super cuquis en cada una de las tres situaciones descritas:

#### Situación inicial:

In [124]:
1 / mu

4.0

#### Situación con servicio de llamada en espera:

In [125]:
cal_R_MM1(lambda_, mu)

p is:  0.8


20.000000000000004

#### Situación con discriminación en las llamadas:

In [126]:
cal_R_priority_nonpreemptive_MM1(np.array([p_super_cuquis, 1-p_super_cuquis])*lambda_, np.array([mu]*2))[0]

p is:  [0.16 0.64]
U is:  [0.   0.16 0.8 ]
X^2 is:  [32. 32.]
S is:  3.2000000000000006
W is:  [ 3.80952381 19.04761905]
R is:  [ 7.80952381 23.04761905]


7.80952380952381

## Problema 5

Una empresa ha recogido datos de las llamadas telefónicas recibidas en los últimos años en horas punta. El tiempo entre llamadas sigue una distribución exponencial con una frecuencia media de 22 llamadas a la hora. El análisis de las conversaciones revela que la duración media de las llamadas es de 4.9 minutos. La empresa tiene 3 operadores atendiendo el teléfono.

In [127]:
lambda_ = 22 / 60
mu = 1 / 4.9
c = 3
print(lambda_, mu, c)

0.36666666666666664 0.2040816326530612 3


### Este sistema se ajusta a un modelo M/M/c

### ¿Cuál es la probabilidad de que ningún operador esté ocupado?

In [128]:
cal_p0_MMC_without_lost(lambda_, mu, c)

p is:  [0.59888889]
p0 is:  [0.14661675]


array([0.14661675])

### ¿Cuál es la probabilidad de que una llamada tenga que esperar por encotnrar a los operadores atendiendo otra llamada? 

In [129]:
cal_wait_probability_MMC_without_lost(lambda_, mu, c)

p is:  0.5988888888888889
p is:  [0.59888889]
p0 is:  [0.14661675]
The p serie is  [array([0.14661675]), array([0.26342142]), array([0.23664024]), array([0.14172121])]


0.35332158458904117

### ¿Cuál es el número medio de operadores ocupados?

In [130]:
def cal_busy_server(lambda_, mu, c):
    p = cal_p(lambda_, mu, c)
    n = c * p
    print("The average of working server is: ", n)
    return n

In [131]:
cal_busy_server(lambda_, mu, c)

p is:  0.5988888888888889
The average of working server is:  1.7966666666666669


1.7966666666666669

### ¿Cuál es el número medio de llamadas en la cola

In [132]:
cal_Q_MMC(lambda_, mu, c)

p is:  0.5988888888888889
p is:  [0.59888889]
p0 is:  [0.14661675]
Q is:  [0.52753555]


array([0.52753555])

### ¿Cuál es el tiempo medio de espera?

In [133]:
cal_Q_MMC(lambda_, mu, c) / lambda_

p is:  0.5988888888888889
p is:  [0.59888889]
p0 is:  [0.14661675]
Q is:  [0.52753555]


array([1.43873332])

## Problema 6

Poisson con unas tasas de llegadas de 13 y 24 trabajos por hora, respectivamente. El tiempo de servicio es exponencial con una tasa de 30 y 72 trabajos por hora para los trabajos A y B, respectivamente. Se considera un modelo de prioridades expulsivas con mayor prioridad a los trabajos A

In [134]:
lambdas = np.array([13, 24])
mus = np.array([30, 72])
print(lambdas, mus)

[13 24] [30 72]


### El modelo sin prioridades sistema se ajusta a un modelo M/M/1

### ¿Cuál es el tiempo medio de respuesta de los trabajos de tipo A?

In [135]:
cal_R_MM1_preemptive(lambdas, mus)[0]

p is:  [0.43333333 0.33333333]
U is:  [0.         0.43333333 0.76666667]
X^2 is:  [0.00222222 0.0003858 ]
left is:  [0.03333333 0.0245098 ]
numerator is:  [0.01444444 0.01907407]
denominator is:  [0.56666667 0.13222222]
right is:  [0.56666667 0.13222222]
R is:  [0.05882353 0.16876751]


0.058823529411764705

### ¿Cuál es el tiempo medio de respuesta de las trabajos de tipo B?

In [136]:
cal_R_MM1_preemptive(lambdas, mus)[1]

p is:  [0.43333333 0.33333333]
U is:  [0.         0.43333333 0.76666667]
X^2 is:  [0.00222222 0.0003858 ]
left is:  [0.03333333 0.0245098 ]
numerator is:  [0.01444444 0.01907407]
denominator is:  [0.56666667 0.13222222]
right is:  [0.56666667 0.13222222]
R is:  [0.05882353 0.16876751]


0.16876750700280108

### ¿Cuál es el tiempo medio de respuesta total?

In [137]:
cal_R_total_MM1_preemptive(lambdas, mus)

weights are : [0.35135135 0.64864865]
p is:  [0.43333333 0.33333333]
U is:  [0.         0.43333333 0.76666667]
X^2 is:  [0.00222222 0.0003858 ]
left is:  [0.03333333 0.0245098 ]
numerator is:  [0.01444444 0.01907407]
denominator is:  [0.56666667 0.13222222]
right is:  [0.56666667 0.13222222]
R is:  [0.05882353 0.16876751]


0.13013854190324775

## Problema 7

Un taller de reparación de vehículos con un funcionamiento continuo (24 horas al día) recibe un promedio de 4 vehículos a la hora. Los vehículos que requieren reparaciones de motor, carrocería o ambas. Los vehículos que requieren simultáneamente a ambas reparaciones son enviados primero a reparación de motor. 

* Los vehículos siguen el siguiente flujo en la empresa: Un clasificador (S1) determina el tipo de avería: motor o carrocería o ambos y envía el vehículo al servicio adecuado. Esta tarea se realiza en un promedio de 6 minutos. Como resultado, se determina que el 10% de los trabajos no son realizables y se descartan; el 35% de los trabajos requieren sólo reparación de carrocería, el 45% sólo de motor y el 10% restante de los dos.

* El servicio de reparación de motor (S2) invierte un promedio de 20 minutos en la reparación. Una vez finalizado el trabajo, los vehículos que requieren reparación de carrocería son enviados al sistema S3, mientras que el resto son enviados a un sistema de preparación del vehículos (S5) para devolvérselo a su propietario (lavado, facturación, etc.).

* El servicio de reparación de carrocería (S3) requiere un promedio de 23 minutos en la reparación. Al final del proceso, los vehículos son enviados al sistema de secado (S4).

* El sistema de secado (S4) requiere un promedio de 11 minutos. Como resultado de este proceso, el 9% de los trabajos son reenviados al sistema de reparación de carrocería, mientras que el resto son enviados al sistema de preparación del vehículo para devolvérselo a su propietario.

* El sistema de preparación del vehículo para devolvérselo a su propietario (S5) requiere un promedio de 7 minutos.

* Se considera que todos los tiempos de proceso de los sistemas se pueden aproximar a una distribución exponencial. El proceso de llegada se puede considerar de Poisson.

Denotamos por $I_i$ la tasa de llegada al servidor i

In [138]:
lambda_ = 4
mus = 1 / np.array([6, 20, 23, 11, 7])
q12 = 0.45
q13 = 0.35
q123 = 0.1
q43 = 0.09
print(lambda_, mus)

4 [0.16666667 0.05       0.04347826 0.09090909 0.14285714]


![Problema7](problema7.jpg)

In [139]:
q23 = q123 / (q12 + q123)
q25 = q12 / (q12 + q123)
q45 = 1 - q43

In [140]:
I1 = lambda_
print("I1=lambda")
I2 = (q12 + q123) * I1
print("I2=", q12+q123, "*I1")
I3 = (q13 * I1 + q23 * I2) / (1 - q43)
print("I3=", q13, "*I1 + ", q23, "*I2 + ", q43, "*I4")
I4 = I3
print("I4=I3")
I5 = q25 * I2 + q45 * I4
print("I5=", q25, "*I2 + ", q45, "*I4")
I = np.array([I1, I2, I3, I4, I5])
print(I)

I1=lambda
I2= 0.55 *I1
I3= 0.35 *I1 +  0.18181818181818182 *I2 +  0.09 *I4
I4=I3
I5= 0.8181818181818181 *I2 +  0.91 *I4
[4.         2.2        1.97802198 1.97802198 3.6       ]


In [141]:
cal_R_total_queueing_network_MM1(I / 60, mus, np.array([lambda_ / 60]))

R is:  [10.         75.         95.13636364 17.25862069 12.06896552]


117.69200626959243

## Problema 8

Una empresa de reparación de aparatos electrónicos recibe un promedio de 17 aparatos por hora y tiene el siguiente esquema de funcionamiento:

* Los clientes son atendidos en la recepción por una persona para una inspección básica del aparato y, si procede, tomar nota de los datos y recoger el aparato. Esta operación necesita un tiempo medio de 2.1 minutos. Como resultado, se determina que el 20% tienen daños de software, el 60% de los aparatos tienen daños en el hardware y el 20% restante no son reparables, por lo que son rechazados.

* Los aparatos con daño de software son revisados por un experto en este tipo de daños que procede a un examen más detallado del aparato (para lo que necesita, en promedio, 10 minutos) y, con una descripción detallada de la reparación necesaria, pasa el aparato al taller de reparación de software.

* Los aparatos con daño de hardware son revisados directamente por otro experto en este tipo de daños que procede a un examen más detallado del aparato (para lo que necesita, en promedio, 6 minutos). El 30% de los aparatos son descartados por irreparables, y, previo aviso al cliente, se destruyen; el 40% son sustituidos por un aparato nuevo, se da el aparato por reparado y se envía al departamento de atención al cliente; el 30% restante son enviados al taller de reparación de hardware.

* El taller de software repara un promedio de 10 aparatos por hora y el taller de hardware repara un promedio de 6 aparatos por hora.

* Una vez finalizada la reparación de un aparato, es enviado al departamento de atención al cliente.

* El departamento de atención al cliente procesa el pago de la reparación y el envío del aparato al cliente. Esta operación requiere un tiempo promedio de 4 minutos.

Por tanto, se puede considerar que el sistema tiene seis servidores:

* S1 : servicio de recepción, que recibe los aparatos con una tasa λ=17 aparatos por hora y necesita un promedio de 2.1 minutos para hacer una revisión básica.

* S2 : servicio de revisión de software, que recibe los aparatos que tienen una avería de software desde el servicio de recepción. Necesita un promedio de 10 minutos para revisar detenidamente cada aparato y lo envía directamente al taller de software.

* S3 : servicio de revisión de hardware, que recibe los aparatos que tienen una avería de hadware desde el servicio de recepción. Necesita un promedio de 6 minutos para revisar detenidamente cada aparato y, según la avería, lo envía al taller de hadware, lo sustituye por uno nuevo o lo destruye por irreparable.

* S4 : taller de software, que recibe los aparatos del servicio de revisión de software ( S2 ), después de repararlo (con una tasa μ4 =10), envía el aparato al servicio de atención al cliente.

* S5 : taller de hadware, que recibe los aparatos del servicio de revisión de hadware ( S2 ), después de repararlo (con una tasa μ5 =6), envía el aparato al servicio de atención al cliente.

* S6 : servicio de atención al cliente, que recibe aparatos del servicio de revisión del hadware ( S3 ) y de los dos talleres ( S4 y S5 ). Tramita el pago de la reparación en un promedio de 4 minutos por aparato.

In [142]:
lambda_ = 17
mus = 1 / np.array([2.1, 10, 6, 1/(10/60), 1/(6/60), 4])
q12 = 0.2
q13 = 0.6
q35 = 0.3
q36 = 0.4
print(lambda_, mus)

17 [0.47619048 0.1        0.16666667 0.16666667 0.1        0.25      ]


![Problema8](problema8.jpg)

In [143]:
I1 = lambda_
print("I1=lambda")
I2 = q12 * I1
print("I2=", q12, "*I1")
I3 = q13 * I1
print("I3=", q13, "*I1")
I4 = I2
print("I4=I2")
I5 = q35 * I3
print("I5=", q35, "*I3")
I6 = q36 * I3 + I4 + I5
print("I6=", q36, "*I3 + I4 + I5")
I = np.array([I1, I2, I3, I4, I5, I6])
print(I)

I1=lambda
I2= 0.2 *I1
I3= 0.6 *I1
I4=I2
I5= 0.3 *I3
I6= 0.4 *I3 + I4 + I5
[17.    3.4  10.2   3.4   3.06 10.54]


In [144]:
cal_R_total_queueing_network_MM1(I / 60, mus, np.array([lambda_]) / 60)

R is:  [   5.18518519   23.07692308 -300.            9.09090909   20.40816327
   13.4529148 ]


-156.36697181860575

# [Prática 2](https://vitaminac.github.io/)

In [145]:
lambda_ = 5000
mu1 = 1 / (4 / 60 / 60)
mu2 = 3 * 60
mu3 = 1 / (5 / 60)
mu_H = 15 * 60
mus = np.array([mu1, mu2, mu3, mu_H])
print(lambda_, mus)

5000 [900. 180.  12. 900.]


In [146]:
I1 = lambda_ / (1 - 0.01)
I2 = (0.15 * 0.99 * I1) / (1 - 0.01)
I3 = (0.2 * 0.99 * I2) / (1 - 0.01)
I_H = 0.85 * 0.99 * I1 + 0.8 * 0.99 * I2 + 0.98 * 0.99 * I3
I_P = 0.02 * 0.99 * I3
I = np.array([I1, I2, I3, I_H, I_P])
print("I in hour is:", I)
assert abs(I_H + I_P - 5000) < 0.00001
T = (1 / I) * 60 * 60
print("T in second is:", T)
print("X in second: ", 1 / mus * 60 *60)

I in hour is: [5.05050505e+03 7.57575758e+02 1.51515152e+02 4.99700000e+03
 3.00000000e+00]
T in second is: [7.12800000e-01 4.75200000e+00 2.37600000e+01 7.20432259e-01
 1.20000000e+03]
X in second:  [  4.  20. 300.   4.]


## La capacidades de cada subsistema para que el nivel de uso sea inferior de 0.8

In [147]:
c = I[:-1] / (0.8 * mus)
print(c)
c = np.ceil(c)
print(c)
c[3] = np.ceil(c[3] / 2) * 2
c

[ 7.01459035  5.26094276 15.78282828  6.94027778]
[ 8.  6. 16.  7.]


array([ 8.,  6., 16.,  8.])

In [148]:
cal_p(I[:-1],mus,c)

p is:  [0.70145903 0.70145903 0.78914141 0.69402778]


array([0.70145903, 0.70145903, 0.78914141, 0.69402778])

### Tiempo medio de inspección total para las maletas.

In [149]:
np.sum(I[:3] / lambda_ * cal_R_MMC(I[:3], mus[:3], c[:3])) * 60 * 60

p is:  [0.70145903 0.70145903 0.78914141]
p is:  [0.70145903 0.70145903 0.78914141]
p0 is:  [3.34060745e-03 1.30843663e-02 2.96894158e-06]
Q is:  [0.64126535 0.79494307 1.05087585]
W is:  [0.00012697 0.00104932 0.00693578]


17.952316832375416

### Tiempo medio de espera en las distintas colas de los sistemas de inspección y en el hipódromo o zonas de acumulación.

In [150]:
cal_W_MMC(I[:-1], mus, c)[:-1] * 60 * 60

p is:  [0.70145903 0.70145903 0.78914141 0.69402778]
p is:  [0.70145903 0.70145903 0.78914141 0.69402778]
p0 is:  [3.34060745e-03 1.30843663e-02 2.96894158e-06 3.56858570e-03]
Q is:  [0.64126535 0.79494307 1.05087585 0.59254751]
W is:  [0.00012697 0.00104932 0.00693578 0.00011858]


array([ 0.45709394,  3.77756948, 24.96881008])

Para 4 hipódromo

In [151]:
cal_W_MMC(I_H / 4, mu_H, 2) * 60 * 60

p is:  0.6940277777777778
p is:  [0.69402778]
p0 is:  [0.18061818]
Q is:  [1.28990589]
W is:  [0.00103254]


array([3.71715927])

### Número de maletas presentes en el sistema

In [152]:
Q = np.concatenate((cal_Q_MMC(I[:-1], mus, c)[:-1], cal_Q_MMC(I[3] / 4, mus[3], 2) * 4), axis=None)
print(Q)
Q_X = (1 / mus) * I[:-1]
print(Q_X)
J =  Q + Q_X
print(J)
np.sum(J)

p is:  [0.70145903 0.70145903 0.78914141 0.69402778]
p is:  [0.70145903 0.70145903 0.78914141 0.69402778]
p0 is:  [3.34060745e-03 1.30843663e-02 2.96894158e-06 3.56858570e-03]
Q is:  [0.64126535 0.79494307 1.05087585 0.59254751]
p is:  0.6940277777777778
p is:  [0.69402778]
p0 is:  [0.18061818]
Q is:  [1.28990589]
[0.64126535 0.79494307 1.05087585 5.15962357]
[ 5.61167228  4.20875421 12.62626263  5.55222222]
[ 6.25293763  5.00369728 13.67713847 10.71184579]


35.64561916939008

Para el modelo 2

In [153]:
mus[:3] /= (0.2*2+0.8)
Q = np.concatenate((cal_Q_MMC(I[:-1], mus, c)[:-1], cal_Q_MMC(I[3] / 4, mus[3], 2) * 4), axis=None)
print(Q)
Q_X = (1 / mus) * I[:-1]
print(Q_X)
J =  Q + Q_X
print(J)
np.sum(J)

p is:  [0.84175084 0.84175084 0.9469697  0.69402778]
p is:  [0.84175084 0.84175084 0.9469697  0.69402778]
p0 is:  [8.33893634e-04 4.15926313e-03 1.10441312e-07 3.56858570e-03]
Q is:  [ 2.93953914  3.22250426 13.71187541  0.59254751]
p is:  0.6940277777777778
p is:  [0.69402778]
p0 is:  [0.18061818]
Q is:  [1.28990589]
[ 2.93953914  3.22250426 13.71187541  5.15962357]
[ 6.73400673  5.05050505 15.15151515  5.55222222]
[ 9.67354588  8.27300931 28.86339056 10.71184579]


57.52179153613749

In [154]:
np.sum(I[:3] / lambda_ * cal_R_MMC(I[:3], mus[:3], c[:3])) * 60 * 60

p is:  [0.84175084 0.84175084 0.9469697 ]
p is:  [0.84175084 0.84175084 0.9469697 ]
p0 is:  [8.33893634e-04 4.15926313e-03 1.10441312e-07]
Q is:  [ 2.93953914  3.22250426 13.71187541]
W is:  [0.00058203 0.00425371 0.09049838]


33.703160936433555